In [1]:
import pandas as pd
import numpy as np
#import preprocessor as p
import csv, re
from nltk.corpus import stopwords
from nltk.tokenize import TweetTokenizer
from nltk.corpus import wordnet
from nltk.corpus import words
import string
import codecs
from sklearn.model_selection import train_test_split
import itertools
from string import digits
from autocorrect import spell 
import wordninja
from nltk.stem import WordNetLemmatizer
import nltk
from sklearn.externals import joblib

In [2]:
def expand_abbrv(input_string):
    input_string = input_string.split(" ")
    slangs = pd.read_csv("../Data/slangs.csv", delimiter=",")
    j = 0
    for _str in input_string:
        _str = re.sub('[^a-zA-Z0-9-_.]', '', _str)
        if(_str.upper() in slangs['abbv'].unique()):
            idx = slangs.index[slangs['abbv'] == _str]
            input_string[j] = slangs['fullform'][idx].str.lower().to_string(index = False)
        j = j + 1
    return (' '.join(input_string))

In [3]:
def expand_contractions(input_string):
    input_string = input_string.split(" ")
    contra = pd.read_csv("../Data/contractions.csv", delimiter=",")
    j = 0
    for _str in input_string:
        if(_str.lower() in contra['contraction'].unique()):

            idx = contra.index[contra['contraction'] == _str]
            input_string[j] = contra['fullform'][idx].str.lower().to_string(index = False)
        j = j + 1
    return (' '.join(input_string))

In [4]:
def remove_hashtags(input_string):
    input_string = input_string.split(" ")
    j = 0
    for _str in input_string:
        if('#' in _str):
            input_string[j] = _str.replace('#','')
        j = j + 1
    return (' '.join(input_string))

In [5]:
embedding_dict = dict()
glove_words = []
#with open('../Dataset/glove.twitter.27B/glove.twitter.27B.25d.txt','rb') as f:
with open('../Data/glove.6B/glove.6B.50d.txt','rb') as f:

    for idx,ln in enumerate(f):
        decoded=False
        line=''
        for cp in ('cp1252', 'cp850','utf-8','utf8'):
            try:
                line = ln.decode(cp)
                decoded=True
                break
            except UnicodeDecodeError:
                pass
        if decoded:
            if(idx==0):
                continue

            split = line.split(' ')

            word = split[0]
            glove_words.append(word)

            features = split[1:]
            features = np.array(
                [float(val) for val in features]
            )
            embedding_dict[word] = features

number_to_word = glove_words
word_to_number = dict((word,idx) for idx,word in enumerate(glove_words))

joblib.dump(embedding_dict,'../Data/rnn_embedding_dict.pkl')
joblib.dump(glove_words,'../Data/rnn_glove_words.pkl')

['Data/rnn_glove_words.pkl']

In [6]:
print(embedding_dict['saint'])

[ 1.2764     2.3994    -1.1009    -1.5421     0.50186   -0.27513
 -0.84622    0.1746    -0.18847   -0.041907   0.51878    0.61113
 -0.17486   -0.92846   -0.32452   -0.40843   -0.19901   -1.1288
 -0.8511     0.55934   -0.92668   -0.18713   -0.94548    0.45426
 -0.44357   -0.79661   -0.94763   -0.20948   -0.84215   -0.0076193
  1.3822     0.6235     0.26689   -0.31898    0.021835  -0.25068
  0.65412    0.21796    0.62775    0.97598    0.82678    0.44615
 -0.022523  -0.92487    0.017586   1.253     -0.83208   -1.4275
  0.3922    -0.6229   ]


In [7]:
def process_data(dataset):

    
    
    #step 1: remove stop words
    stop = set(stopwords.words('english'))
    not_stop = ["don't","weren't","doesn't","isn't","aren't","not","needn't","won't","hasn't","mightn't","didn't","haven't","hadn't","shouldn't","wasn't","mustn't","couldn't"]
    stop = (set(stop).difference(not_stop))
    dataset = dataset.apply(lambda x:' '.join([word for word in x.split() if word not in (stop)]))

    #step 2: no http links, remove @ words
    dataset = dataset.apply(lambda x:' '.join([word for word in x.split() if '@' not in word]))

    #step 3: remove hastags
    dataset = dataset.apply(lambda x: remove_hashtags(x))

    #step 4: expand contractions
    dataset = dataset.apply(lambda x: expand_contractions(x))

    #step 5: expand abbreviations
    dataset = dataset.apply(lambda x: expand_abbrv(x))
    
    #step 6: remove punctuations
    punc = string.punctuation
    dataset = dataset.apply(lambda x: list(x))
    dataset = dataset.apply(lambda x: ''.join([o for o in x if not o in punc]).split())
    dataset = dataset.apply(lambda x: ' '.join(x))

    #step 7: remove nos.
    remove_digits = str.maketrans('', '', digits)
    dataset = dataset.str.translate(remove_digits)
    
    #step 8: shorten elongated words
    dataset = dataset.apply(lambda x: ''.join(''.join(s)[:2] for _, s in itertools.groupby(x)))

    #step 9: all lower case
    dataset = dataset.str.lower()

    #step 10: remove punctuations
    punc = string.punctuation
    dataset = dataset.apply(lambda x: list(x))
    dataset = dataset.apply(lambda x: ''.join([o for o in x if not o in punc]).split())
    dataset = dataset.apply(lambda x: ' '.join(x))
    
    #processing 2: separate joined words and spell check

    lemmatizer = WordNetLemmatizer()
    english_vocab = set(w.lower() for w in nltk.corpus.words.words())
    processed_dataset = []
    missing_words = []
    number_missing = 0
    for el_id,el in enumerate(dataset):
        newL = []
        word_list = el.split(' ')
        for idx,word in enumerate(word_list):
            if not word in embedding_dict:
                b = spell(word)
                if ((lemmatizer.lemmatize(b) in words.words()) and (not word in words.words())):
                    newL.append(b)
                else:
                    a = wordninja.split(word)
                    flag = False

                    for item in a:
                        if not item in words.words():
                            flag = True

                    if (flag == False and len(a)>1):
                       newL.append(" ".join(a))
                    else:
                        newL.append(word)
                number_missing = number_missing + 1
                missing_words.append(word)
            else:
                newL.append(word)

        newE = ' '.join(newL)
        processed_dataset.append(newE)
        
    processed_dataset = np.asarray(processed_dataset) 
    number_missing_2 = 0
    missing_words_2 = []

    idx_with_missing_word  = []     
    for el_id,el in enumerate(processed_dataset):
        changed = False
        word_list = el.split(' ')
        new_sentence = []
        for word in word_list:
            if not word in embedding_dict:
                changed = True
            else:
                new_sentence.append(word)

        if(changed == True):
            processed_dataset[el_id] = " ".join(new_sentence)

    for el_id,el in enumerate(processed_dataset):
        word_list = el.split(' ')
        for word in word_list:
            if not word in embedding_dict:
                number_missing_2 = number_missing_2 + 1
                missing_words_2.append(word)
                idx_with_missing_word.append(el_id)

    dataset = dataset.apply(lambda x:' '.join([word for word in x.split() if word not in (stop)]))            
                
    print(processed_dataset.shape)
    return processed_dataset

In [8]:
train_files = ['../Data/Raw/joy_train.txt','../Data/Raw/surprise_train.txt','../Data/Raw/anger_train.txt','../Data/Raw/fear_train.txt','../Data/Raw/sadness_train.txt']
processed_files = ['../Data/Processed/processed_joy_train.txt','../Data/Processed/processed_surprise_train.txt','../Data/Processed/processed_anger_train.txt','../Data/Processed/processed_fear_train.txt','Data/Processed/processed_sadness_train.txt']
for idx, iF in enumerate(train_files):   
    dataset = pd.read_csv(iF, sep="\t",header=None)
    print(dataset.shape)
    if(idx!=1):
        dataset = dataset.loc[dataset[3]>=0.5]
        dataset = process_data(dataset[1])
    else:
        dataset = process_data(dataset[0])
    dataset_pd = pd.DataFrame(data=dataset) 
    dataset_pd.to_csv(processed_files[idx], index=False,header = False)

(1616, 4)
(821,)
(212, 1)
(212,)
(1701, 4)
(834,)
(2252, 4)
(1115,)
(1533, 4)
(764,)
